# Best ligand

choosing the ligand with the best fit after qvina docking for all of them

In [ ]:
import os
import tqdm
import numpy as np
import pandas as pd
import nglview as nv
import MDAnalysis as mda

In [ ]:
folder = "data/out/"
files = os.listdir(folder)

In [ ]:
results = {}
for name in tqdm.tqdm(files):
    if ".pdbqt" in name:
        with open(folder + name, "r") as f:
            model = 1
            for l in f.readlines():
                if "REMARK VINA RESULT" in l:
                    m = model
                    if results.get(name[:-10]) is None:
                        results[name[:-10]] = {m:float(l.split()[3])}
                    else:
                        results[name[:-10]][m] = float(l.split()[3])
                    model += 1


In [ ]:
df = pd.DataFrame.from_dict(results, orient='index')
df['min'] = df.min(axis=1)
df_sf = df.sort_values('min').head(10)
df_sf.to_csv('report/figures/best_fits.csv',header=False)
df_sf

In [ ]:
best_ligand = df_sf.iloc[2].name
print(best_ligand)
with open(f'{folder}{best_ligand}_out.pdbqt', "r") as f:
    print(f.read())


In [ ]:
view = nv.show_file(f'data/fda_pdbqt/{best_ligand}.pdbqt',ext='pdbqt')
# view.clear()
# view.add_representation('cartoon', selection='protein')
# view.camera = 'orthographic'  # Optional: Sets the camera to orthographic for clearer views without perspective distortion
view.parameters = {"fogNear": 0, "fogFar": 1000} 
view

In [ ]:
# view.download_image("best_ligand.png",factor=10,trim=True,transparent=True)

In [ ]:
df.loc['saquinavir']

In [ ]:
with open(f'{folder}saquinavir_out.pdbqt', "r") as f:
    print(f.read())

In [ ]:
view = nv.show_file(f'data/saquinavir.pdbqt',ext='pdbqt')
# view.clear()
# view.add_representation('label',selection='protein',labelType='atom name')
# view.add_representation('cartoon', selection='protein')
# view.camera = 'orthographic'  # Optional: Sets the camera to orthographic for clearer views without perspective distortion
view.parameters = {"fogNear": 0, "fogFar": 1000} 
view

In [ ]:
# view.download_image("figures/saquinavir_nglview.png",factor=10,trim=True,transparent=True)

# MD

In [ ]:
# protein
view = nv.show_file(f'md/pro.pdb',ext='pdb')
view.clear()
# view.add_representation('label',selection='protein')
# view.add_representation('spacefill',selection='protein',sphereDetail=0)
view.add_representation('surface',surfaceType='av',colorScheme='chainname')
# view.camera = 'orthographic'  # Optional: Sets the camera to orthographic for clearer views without perspective distortion
view.parameters = {"fogNear": 0, "fogFar": 1000} 
view

In [ ]:
# view.download_image("hiv1_nglview.png",factor=10,trim=True,transparent=True)

In [ ]:
# protein
view = nv.show_file(f"data/1hxb.pdb",ext='pdb')
view.clear()
view.add_representation('licorice',selection='protein',color='white')
view.add_representation('hyperball',selection='ligand',colorScheme='densityfit')
# view.add_representation('base',selection='ligand')
view.add_representation('distances',selection='ligand')
view.add_distance()
# view.add_representation('spacefill',selection='protein',sphereDetail=0)
# view.add_representation('surface',surfaceType='av',colorScheme='chainname')
# view.camera = 'orthographic'  # Optional: Sets the camera to orthographic for clearer views without perspective distortion
view.parameters = {"fogNear": 0, "fogFar": 1000} 
view

In [ ]:
view.download_image("distances_nglview.png",factor=10,trim=True,transparent=True)

## Saquinavir md

In [ ]:
# Load the molecular dynamics universe
u = mda.Universe('test_md/md001.tpr', 'test_md/md001.xtc')

# Create a NGLview widget
view = nv.show_mdanalysis(u)
view.clear()
view.add_representation('cartoon', selection='protein')  # Adds a cartoon representation of the protein
view.add_representation('ball+stick', selection='turn')  # Adjust selection as needed
# view.add_representation('licorice', selection='seg_0_system1')
# Display the widget
view

## Ligand fda1700 md

In [ ]:
# Load the molecular dynamics universe
u = mda.Universe('md/md001.tpr', 'md/md001.xtc')

# Create a NGLview widget
view = nv.show_mdanalysis(u)
view.clear()
view.add_representation('cartoon')  # Adds a cartoon representation of the protein
view.add_representation('ball+stick', selection='ligand')  # Adjust selection as needed

# Display the widget
view

# Data Analysis

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True

In [ ]:
# Function to read an XVG file and return the data
def read_xvg(filename,verbose=False):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            if not line.startswith(('@', '#', '&')):  # Skip comments and metadata
                split_line = line.strip().split()
                data.append([float(x) for x in split_line])  # Convert strings to floats
            elif verbose:
                print(line)
    return np.array(data).transpose()  # This transposes the data, unpacking it into columns

In [ ]:
xvg_file = 'md/rmsd.xvg'
lig_data = read_xvg(xvg_file,verbose=False)
xvg_file = 'test_md/rmsd.xvg'
saq_data = read_xvg(xvg_file,verbose=False)

fig = plt.figure(figsize=(4,3), dpi=100)
ax = fig.add_subplot(1,1,1)
ax.plot(lig_data[0], lig_data[1],c="#3250e5",lw=0.5,label='fda_1700')
ax.plot(saq_data[0], saq_data[1],c="#dd1010",lw=0.5,label='saquinavir')
ax.legend()
ax.set_xlabel(r'Time (ps)', fontsize=11)  
ax.set_ylabel(r'RMSD (nm)') 
# ax.set_title(r'\textbf{RMSD} Lig after lsq fit to Protein') 
fig.savefig('report/figures/rmsd.svg', format='svg', transparent=True, bbox_inches='tight', pad_inches=0)

In [ ]:
fig = plt.figure(figize=(4,3), dpi=100)
ax = fig.add_subplot(1,1,1)
ax.plot(data[0], data[1],c="#3250e5",lw=0.5)
ax.set_xlabel(r'Time (ps)', fontsize=11)  
ax.set_ylabel(r'RMSD (nm)') 
# ax.set_title(r'\textbf{RMSD} Lig after lsq fit to Protein') 